In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
timeseries_monthly = pd.read_csv("dataset/raw_data/timeseries/time_series_monthly_adjusted.csv")
income_statement = pd.read_csv("dataset/raw_data/roic_data/isq.csv")
balance_sheet = pd.read_csv("dataset/raw_data/roic_data/bsq.csv")
cash_flow_statement = pd.read_csv("dataset/raw_data/roic_data/cfq.csv")
#earnings = pd.read_csv("dataset/raw_data/fundamentals/earnings.csv")
overview = pd.read_csv("dataset/raw_data/fundamentals/overview.csv")

income_statement["fillingDate"] = (
    income_statement["fillingDate"]
    .fillna(
        income_statement["date"]
        .apply(lambda x: "-".join([x.split("/")[2], x.split("/")[0], x.split("/")[1]])))
)
balance_sheet["fillingDate"] = (
    balance_sheet["fillingDate"]
    .fillna(
        balance_sheet["date"]
        .apply(lambda x: "-".join([x.split("/")[2], x.split("/")[0], x.split("/")[1]])))
)
cash_flow_statement["fillingDate"] = (
    cash_flow_statement["fillingDate"]
    .fillna(
        cash_flow_statement["date"]
        .apply(lambda x: "-".join([x.split("/")[2], x.split("/")[0], x.split("/")[1]])))
)

timeseries_monthly["timestamp"] = pd.to_datetime(timeseries_monthly["timestamp"])
income_statement["fillingDate"] = pd.to_datetime(income_statement["fillingDate"])
balance_sheet["fillingDate"] = pd.to_datetime(balance_sheet["fillingDate"])
cash_flow_statement["fillingDate"] = pd.to_datetime(cash_flow_statement["fillingDate"])
#earnings["reportedDate"] = pd.to_datetime(earnings["reportedDate"])
overview = overview[["symbol", "Sector"]]


all_dfs = [timeseries_monthly, income_statement, balance_sheet, cash_flow_statement]
merge_columns = ["timestamp", "fillingDate", "fillingDate", "fillingDate"]

In [10]:
def merge_related(dfs: list[pd.DataFrame], merge_c: list[str], by_c: str="symbol") -> pd.DataFrame:
    merged = dfs[0]
    for i in range(len(dfs)-1):
        merged = (pd.merge_asof(
                merged.sort_values([merge_c[0]]),
                dfs[i+1].sort_values([merge_c[i+1]]), 
                allow_exact_matches=True,
                left_on=merge_c[0],
                right_on=merge_c[i+1],
                by=by_c
            )
            .dropna(subset=[merge_c[i+1]])
            .drop("fillingDate", axis=1)
        )
    
    return merged


# merged = (
#     pd.merge_asof(
#         timeseries_monthly.sort_values(["timestamp"]),
#         income_statement.sort_values(["fiscalDateEnding"]), 
#         allow_exact_matches=True,
#         left_on="timestamp",
#         right_on="fiscalDateEnding",
#         by="symbol"
#     )
#     .dropna(subset=["fiscalDateEnding"])
#     .drop("fiscalDateEnding", axis=1)
# )

merged = merge_related(all_dfs, merge_columns)

merged = merged.merge(overview, on="symbol")


merged = merged.sort_values(['symbol', 'timestamp'], ascending=[True, False])



merged.head()

,symbol,timestamp,open,high,low,close,adjusted close,volume,dividend amount,date_x,...,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,Sector
95078,A,2023-06-09,116.97,119.760,114.1600,115.79,115.7900,19163443,0.000,01/31/2023,...,22000000.0,197000000.0,1.253000e+09,1.056000e+09,296000000.0,-76000000.0,220000000.0,https://www.sec.gov/Archives/edgar/data/109087...,https://www.sec.gov/Archives/edgar/data/109087...,LIFE SCIENCES
95077,A,2023-05-31,136.02,137.160,113.2801,115.67,115.6700,46200248,0.000,01/31/2023,...,22000000.0,197000000.0,1.253000e+09,1.056000e+09,296000000.0,-76000000.0,220000000.0,https://www.sec.gov/Archives/edgar/data/109087...,https://www.sec.gov/Archives/edgar/data/109087...,LIFE SCIENCES
95076,A,2023-04-28,137.42,141.480,128.1100,135.43,135.4300,27776452,0.225,01/31/2023,...,22000000.0,197000000.0,1.253000e+09,1.056000e+09,296000000.0,-76000000.0,220000000.0,https://www.sec.gov/Archives/edgar/data/109087...,https://www.sec.gov/Archives/edgar/data/109087...,LIFE SCIENCES
95075,A,2023-03-31,138.58,145.435,128.2200,138.34,138.1146,38086236,0.000,01/31/2023,...,22000000.0,197000000.0,1.253000e+09,1.056000e+09,296000000.0,-76000000.0,220000000.0,https://www.sec.gov/Archives/edgar/data/109087...,https://www.sec.gov/Archives/edgar/data/109087...,LIFE SCIENCES
95074,A,2023-02-28,153.31,157.310,138.7200,141.97,141.7387,23987443,0.000,10/31/2022,...,-14000000.0,-17000000.0,1.056000e+09,1.073000e+09,448000000.0,-70000000.0,378000000.0,https://www.sec.gov/Archives/edgar/data/109087...,https://www.sec.gov/Archives/edgar/data/109087...,LIFE SCIENCES


In [11]:
ratios = [elem for elem in list(merged.columns) if "ratio" in elem or "Ratio" in elem]
totals = [elem for elem in list(merged.columns) if "total" in elem or "Total" in elem]
final = ["symbol", "timestamp", "adjusted close", "freeCashFlow", "eps", "Sector"]

final.extend(ratios)
final.extend(totals)

In [12]:
merged_final = merged[final]

In [ ]:
for column in merged.columns:
    count = (
        merged[column]
        .isna()
        .sum()
    )

    if count:
        print(f"For column {column} there are {count} missing values!")
        count_symbol = merged[merged[column].isna()]


        print(f"There are missing values for {count_symbol['symbol'].nunique()} companies\n\n")


print(len(merged))
# count_totalAssets = (
#    merged["totalAssets"]
#    .isna()
#    .groupby(balance_sheet["symbol"])
#    .sum()
#    .reset_index(name='count')
#    .sort_values("count", ascending=True)
# )

In [13]:
merged.to_csv("dataset/transformed_data/roic_data.csv")

In [14]:
merged_final.to_csv("dataset/transformed_data/initial_train_data.csv")